In [6]:
#! /usr/bin/env python3

from pymarc import MARCReader
import os
import csv

In [2]:
with open('source_data/MARCDATA.MRC', 'rb') as f:
    reader = MARCReader(f)
    all_records = list()
    for record in reader:
        all_records.append(record)

In [8]:
# an example of one record
# all_records[0].as_dict()

In [4]:
def lookup_uid(record):
    return record.get_fields('001')[0].value().replace('AAI','')

In [10]:
lookup_uid(all_records[0])

'0000418'

In [ ]:
# test of all uids in marc file match a pdf on U drive
# short answer: they al do

# pdf_not_on_U = list()

# for record in all_records:
#     uid = lookup_uid(record)
#     if os.path.isfile('/media/francis/U/ProquestDissertations/UnrestrictedTheses/{}.pdf'.format(uid)):
#         continue
#     pdf_not_on_U.append(uid)

# print(pdf_not_on_U)
